* Tutorial: https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Linear_Mixed_Effects_Models.ipynb

In [20]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import json


import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_probability as tfp


tfd = tfp.distributions
tfb = tfp.bijectors

dtype = tf.float64

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
def load_insteval():
    url = ('https://raw.github.com/vincentarelbundock/Rdatasets/master/csv/'
         'lme4/InstEval.csv')
    with requests.Session() as s:
        download = s.get(url)
        f = download.content.decode().splitlines()

    iterator = csv.reader(f)
    columns = next(iterator)[1:]
    x_train = np.array([row[1:] for row in iterator], dtype=np.int)
    metadata = {'columns': columns}
    return x_train, metadata


In [ ]:
data, metadata = load_insteval()
data = pd.DataFrame(data, columns=metadata['columns'])
data = data.rename(columns={'s': 'students',
                            'd': 'instructors',
                            'dept': 'departments',
                            'y': 'ratings'})
data['students'] -= 1  # start index by 0
# Remap categories to start from 0 and end at max(category).
data['instructors'] = data['instructors'].astype('category').cat.codes
data['departments'] = data['departments'].astype('category').cat.codes

train = data.sample(frac=0.8)
test = data.drop(train.index)

train.head()

In [ ]:
get_value = lambda dataframe, key, dtype: dataframe[key].values.astype(dtype)
features_train = {
    k: get_value(train, key=k, dtype=np.int32)
    for k in ['students', 'instructors', 'departments', 'service']}
labels_train = get_value(train, key='ratings', dtype=np.float32)

features_test = {k: get_value(test, key=k, dtype=np.int32)
                 for k in ['students', 'instructors', 'departments', 'service']}
labels_test = get_value(test, key='ratings', dtype=np.float32)

In [ ]:
num_students = max(features_train['students']) + 1
num_instructors = max(features_train['instructors']) + 1
num_departments = max(features_train['departments']) + 1
num_observations = train.shape[0]

print("Number of students:", num_students)
print("Number of instructors:", num_instructors)
print("Number of departments:", num_departments)
print("Number of observations:", num_observations)

In [ ]:
class LinearMixedEffectModel(tf.Module):
    def __init__(self):
        # Set up fixed effects and other parameters.
        # These are free parameters to be optimized in E-steps
        self._intercept = tf.Variable(0., name="intercept")            # alpha in eq
        self._effect_service = tf.Variable(0., name="effect_service")  #  beta in eq
        self._stddev_students = tfp.util.TransformedVariable(
            1., bijector=tfb.Exp(), name="stddev_students")            # sigma in eq
        self._stddev_instructors = tfp.util.TransformedVariable(
            1., bijector=tfb.Exp(), name="stddev_instructors")         # sigma in eq
        self._stddev_departments = tfp.util.TransformedVariable(
            1., bijector=tfb.Exp(), name="stddev_departments")         # sigma in eq

    def __call__(self, features):
        model = tfd.JointDistributionSequential([
          # Set up random effects.
          tfd.MultivariateNormalDiag(
              loc=tf.zeros(num_students),
              scale_identity_multiplier=self._stddev_students),
          tfd.MultivariateNormalDiag(
              loc=tf.zeros(num_instructors),
              scale_identity_multiplier=self._stddev_instructors),
          tfd.MultivariateNormalDiag(
              loc=tf.zeros(num_departments),
              scale_identity_multiplier=self._stddev_departments),
          # This is the likelihood for the observed.
          lambda effect_departments, effect_instructors, effect_students: tfd.Independent(
              tfd.Normal(
                  loc=(self._effect_service * features["service"] +
                      tf.gather(effect_students, features["students"], axis=-1) +
                      tf.gather(effect_instructors, features["instructors"], axis=-1) +
                      tf.gather(effect_departments, features["departments"], axis=-1) +
                      self._intercept),
                  scale=1.),
                  reinterpreted_batch_ndims=1)
        ])

        # To enable tracking of the trainable variables via the created distribution,
        # we attach a reference to `self`. Since all TFP objects sub-class
        # `tf.Module`, this means that the following is possible:
        # LinearMixedEffectModel()(features_train).trainable_variables
        # ==> tuple of all tf.Variables created by LinearMixedEffectModel.
        model._to_track = self
        return model

In [ ]:
lmm_jointdist = LinearMixedEffectModel()
# Conditioned on feature/predictors from the training data
lmm_train = lmm_jointdist(features_train)

In [ ]:
lmm_train.trainable_variables


In [ ]:
lmm_train.resolve_graph()


In [ ]:
target_log_prob_fn = lambda *x: lmm_train.log_prob(x + (labels_train,))
trainable_variables = lmm_train.trainable_variables
current_state = lmm_train.sample()[:-1]

In [ ]:
hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=target_log_prob_fn,
    step_size=0.015,
    num_leapfrog_steps=3)
kernel_results = hmc.bootstrap_results(current_state)

@tf.function(autograph=False, experimental_compile=True)
def one_e_step(current_state, kernel_results):
    next_state, next_kernel_results = hmc.one_step(
      current_state=current_state,
      previous_kernel_results=kernel_results)
    return next_state, next_kernel_results

optimizer = tf.optimizers.Adam(learning_rate=.01)

# Set up M-step (gradient descent).
@tf.function(autograph=False, experimental_compile=True)
def one_m_step(current_state):
    with tf.GradientTape() as tape:
        loss = -target_log_prob_fn(*current_state)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    return loss


In [ ]:
num_warmup_iters = 1000
num_iters = 1500
num_accepted = 0
effect_students_samples = np.zeros([num_iters, num_students])
effect_instructors_samples = np.zeros([num_iters, num_instructors])
effect_departments_samples = np.zeros([num_iters, num_departments])
loss_history = np.zeros([num_iters])

In [ ]:
for t in range(num_warmup_iters):
  current_state, kernel_results = one_e_step(current_state, kernel_results)
  num_accepted += kernel_results.is_accepted.numpy()
  if t % 500 == 0 or t == num_warmup_iters - 1:
    print("Warm-Up Iteration: {:>3} Acceptance Rate: {:.3f}".format(
        t, num_accepted / (t + 1)))

num_accepted = 0  # reset acceptance rate counter

# Run training.
for t in range(num_iters):
  # run 5 MCMC iterations before every joint EM update
  for _ in range(5):
    current_state, kernel_results = one_e_step(current_state, kernel_results)
  loss = one_m_step(current_state)
  effect_students_samples[t, :] = current_state[0].numpy()
  effect_instructors_samples[t, :] = current_state[1].numpy()
  effect_departments_samples[t, :] = current_state[2].numpy()
  num_accepted += kernel_results.is_accepted.numpy()
  loss_history[t] = loss.numpy()
  if t % 500 == 0 or t == num_iters - 1:
    print("Iteration: {:>4} Acceptance Rate: {:.3f} Loss: {:.3f}".format(
        t, num_accepted / (t + 1), loss_history[t]))

In [ ]:
plt.plot(loss_history)
plt.ylabel(r'Loss $-\log$ $p(y\mid\mathbf{x})$')
plt.xlabel('Iteration')
plt.show()

In [ ]:
for i in range(7):
  plt.plot(effect_instructors_samples[:, i])

plt.legend([i for i in range(7)], loc='lower right')
plt.ylabel('Instructor Effects')
plt.xlabel('Iteration')
plt.show()

# Statsmodels
* https://www.kaggle.com/ojwatson/mixed-models

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [8]:
# Formula indicates mean weight as a linear function of time. Random intercept for each pig. 
data = sm.datasets.get_rdataset('dietox', 'geepack').data
md = smf.mixedlm("Weight ~ Time", data, groups=data["Pig"], re_formula="~Time")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

           Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  Weight    
No. Observations:  861      Method:              REML      
No. Groups:        72       Scale:               6.0372    
Min. group size:   11       Log-Likelihood:      -2217.0475
Max. group size:   12       Converged:           Yes       
Mean group size:   12.0                                    
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        15.739    0.550 28.603 0.000 14.660 16.817
Time              6.939    0.080 86.925 0.000  6.783  7.095
Group Var        19.503    1.561                           
Group x Time Cov  0.294    0.153                           
Time Var          0.416    0.033                           



In [9]:
rmse = mdf.fittedvalues

0       21.957902
1       28.806111
2       35.654320
3       42.502529
4       49.350737
          ...    
856     72.989071
857     80.435264
858     87.881457
859     95.327649
860    102.773842
Length: 861, dtype: float64

# Stats models on the FQI part

In [27]:
with open('train_tuples.json', 'r') as f:
    train_dict = json.load(f)
with open('test_tuples.json', 'r') as f:
    test_dict = json.load(f)

In [28]:
train_df = pd.DataFrame.from_dict(train_dict)
test_df = pd.DataFrame.from_dict(test_dict)
train_df.columns

Index(['a0', 'a1', 'r', 'ds', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7',
       's8', 's9'],
      dtype='object')

In [25]:
y = train_df['r']
X = train_df[['a0', 'a1', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9']]
groups = train_df['ds']s

In [31]:
model = sm.MixedLM(endog=y, exog=X, groups=groups)
result = model.fit()

/Users/aishwaryamandyam/anaconda3/envs/tf/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/Users/aishwaryamandyam/anaconda3/envs/tf/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2202: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
/Users/aishwaryamandyam/anaconda3/envs/tf/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/aishwaryamandyam/anaconda3/envs/tf/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [29]:
testX = test_df[['a0', 'a1', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9']]
result.predict(exog=testX)

0        4.421987
1        9.033591
2       26.392287
3       28.386333
4       35.480719
          ...    
1995    31.096184
1996    14.359744
1997    21.499843
1998     6.120035
1999    15.712445
Length: 2000, dtype: float64